<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dc7bc1cabf1ba69ebf979dd3c2deaafe2c7c70fef10f5ce7a18f07d861348a27
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 14:45:52
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 27.29 K (0.2%)
Current PnL: -28.38 L (-18.33%)
CY Booked + Current PnL: -14.09 L (-9.1%)
-------------------
Total profit:  1.38 L
Total loss:  -29.76 L
-------------------
Total Booked + Current PnL: 12.89 L (10.08%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.01 L (66.49%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.12,-16.71,20.17,0.09,16930.0,-16839.0,83938.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.28,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.41,-12.11,13.81,0.02,21186.0,-21146.0,153413.0,133.82,53.0,M-SC,15.85,234.0,-1.00,1.14,33.44,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.84,-1.21,23.54,22.05,22786.0,-1185.0,96795.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.70,XR,ATH,FINANCE
82,VOLTAS,1530.00,1.25,7.20,11.65,19.69,23946.0,13803.0,205545.0,-3.29,53.0,X-MC,2.59,78.0,0.58,1.52,14.49,XY25,NTT,AC
43,ITC,452.00,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,4.80,-15.67,74.26,46.95,78466.0,-19633.0,105664.0,-22.21,41.0,H-SC,6.80,154.0,-0.25,0.78,32.37,MH,ATH,POWER
10,BANDHANBNK,400.00,3.32,-27.79,166.45,92.42,334832.0,-77398.0,201161.0,41.52,51.0,H-SC,8.71,170.0,-0.23,1.49,16.25,XY24,NTT,BANKS
22,DIXON,18931.72,2.84,-14.22,41.74,21.58,50177.0,-19926.0,120213.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.15,X40N,ATH,IT
46,JSWINFRA,342.00,2.13,-8.12,25.41,15.23,46773.0,-16275.0,184072.0,-21.45,46.0,X-MC,8.86,66.0,-0.35,1.36,21.51,X40N,NTT,REALTY
52,NESTLEIND,1377.00,1.81,9.05,11.32,21.40,32767.0,24032.0,289458.0,1.24,46.0,X-LC,5.44,12.0,0.73,2.15,16.30,XY25,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,-2.75,-45.05,146.15,35.27,91198.0,-51149.0,62400.0,-57.75,31.0,L-SC,11.08,270.0,-0.56,0.46,52.73,XR,NTT,HOTELS
65,SIEMENS,4671.5,-2.24,-16.04,49.49,25.51,77328.0,-29845.0,156250.0,-4.14,42.0,H-LC,4.20,49.0,-0.39,1.16,15.14,AR,ATH,ELECTRICAL
57,RAJOOENG,143.1,-2.14,-38.07,125.43,39.61,79623.0,-39020.0,63480.0,-58.14,24.0,H-SC,28.08,135.0,-0.49,0.47,0.00,AR,ATH,MISC
33,HINDUNILVR,2922.0,-1.88,-12.00,29.17,13.67,65988.0,-30843.0,226220.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.80,XY25,NTT,FMCG
63,SFL,1287.0,-1.71,-43.12,122.65,26.63,182917.0,-113081.0,149137.0,12.97,28.0,M-SC,11.10,239.0,-0.62,1.11,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.20,-6.27,121.83,107.91,168588.0,-9264.0,138380.0,-23.79,41.0,M-SC,12.49,216.0,-0.05,1.03,3.18,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.41,-12.11,13.81,0.02,21186.0,-21146.0,153413.0,133.82,53.0,M-SC,15.85,234.0,-1.00,1.14,33.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.12,-16.71,20.17,0.09,16930.0,-16839.0,83938.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.28,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.19,-25.35,52.00,13.47,104682.0,-68355.0,201312.0,-69.28,31.0,H-SC,4.70,158.0,-0.65,1.49,2.36,XY24,NTT,PAINTS
66,SIS,528.00,0.12,-25.41,62.56,21.26,51815.0,-28208.0,82824.0,1952.33,44.0,H-SC,4.78,166.0,-0.54,0.61,11.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.35,6.71,61.43,72.27,111076.0,11370.0,180818.0,-8.35,68.0,M-LC,2.02,99.0,0.1,1.34,13.3,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.35,6.71,61.43,72.27,111076.0,11370.0,180818.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.30,XR,NTT,IT
38,INDIAMART,4810.62,-0.52,-4.30,116.00,106.72,136922.0,-5300.0,118036.0,-52.54,30.0,H-SC,2.32,139.0,-0.04,0.87,18.08,AR,ATH,MISC
50,MASFIN,398.61,0.84,-1.21,23.54,22.05,22786.0,-1185.0,96795.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.70,XR,ATH,FINANCE
25,FINCABLES,1641.55,0.20,-6.27,121.83,107.91,168588.0,-9264.0,138380.0,-23.79,41.0,M-SC,12.49,216.0,-0.05,1.03,3.18,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.48,-2.25,38.75,35.63,79382.0,-4718.0,204856.0,-15.85,41.0,H-MC,3.11,119.0,-0.06,1.52,14.05,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.23,-21.81,89.85,48.45,140870.0,-43722.0,156784.0,-27.39,24.0,X-MC,12.70,64.0,-0.31,1.16,2.08,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-1.88,-12.00,29.17,13.67,65988.0,-30843.0,226220.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.80,XY25,NTT,FMCG
53,PAGEIND,51605.08,-0.50,-9.91,39.93,26.07,58905.0,-16220.0,147520.0,-33.64,31.0,X-MC,12.41,55.0,-0.28,1.09,0.50,X40,ATH,APPARELS
75,TMPV,600.00,0.40,-17.52,72.39,42.18,165030.0,-48437.0,227973.0,-25.52,31.0,X-LC,6.36,3.0,-0.29,1.69,1.35,XY24,NTT,AUTO
23,DMART,5391.45,0.03,-8.94,40.70,28.12,77981.0,-18807.0,191600.0,-21.41,32.0,X-LC,5.22,19.0,-0.24,1.42,12.58,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.11,-80.19,529.98,24.79,264402.0,-201967.0,49889.0,-81.09,45.0,X-SC,0.94,86.0,-0.76,0.37,21.96,X40N,NTT,MISC
43,ITC,452.0,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG
8,AWL,485.0,0.38,-21.40,95.29,53.50,226003.0,-64577.0,237174.0,-61.97,35.0,X-MC,2.09,58.0,-0.29,1.76,5.34,XY24,NTT,FMCG
36,ICICIPRULI,790.0,1.67,7.18,22.20,30.97,42052.0,12686.0,189424.0,-18.05,64.0,X-MC,2.16,75.0,0.30,1.40,20.64,X40,ATH,INSURANCE
3,ACC,3906.0,-0.31,-25.47,120.25,64.15,213251.0,-60611.0,177340.0,-56.31,33.0,X-SC,2.28,82.0,-0.28,1.31,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.31,-25.47,120.25,64.15,213251.0,-60611.0,177340.0,-56.31,33.0,X-SC,2.28,82.0,-0.28,1.31,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.08,-0.50,-9.91,39.93,26.07,58905.0,-16220.0,147520.0,-33.64,31.0,X-MC,12.41,55.0,-0.28,1.09,0.50,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.08,-38.13,118.52,35.20,94359.0,-49056.0,79614.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.13,X40,NTT,FOOTWEAR
54,PGHH,17907.65,1.28,-4.97,40.75,33.76,77769.0,-9975.0,190845.0,-32.91,43.0,X-MC,5.31,57.0,-0.13,1.41,1.28,X40,ATH,FMCG
31,HAVELLS,2069.16,0.41,-9.84,46.95,32.48,138832.0,-32290.0,295701.0,-14.34,41.0,X-MC,5.69,67.0,-0.23,2.19,1.31,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.47,-29.52,102.68,42.84,47043.0,-19191.0,45815.0,-53.72,42.0,X-SC,36.19,83.0,-0.41,0.34,3.65,XY24,NTT,MISC
15,CAMS,4762.00,-0.11,-80.19,529.98,24.79,264402.0,-201967.0,49889.0,-81.09,45.0,X-SC,0.94,86.0,-0.76,0.37,21.96,X40N,NTT,MISC
59,RELAXO,1176.00,-0.30,-47.48,190.01,52.31,144852.0,-68926.0,76234.0,-44.05,43.0,X-SC,6.65,91.0,-0.48,0.56,2.04,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.08,-38.13,118.52,35.20,94359.0,-49056.0,79614.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.13,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.84,-14.22,41.74,21.58,50177.0,-19926.0,120213.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.15,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.74,-12.63,36.53,19.28,110411.0,-43708.0,302248.0,-23.95,63.0,X-LC,4.93,1.0,-0.40,2.24,11.74,X40,ATH,IT
41,INFY,2275.00,-0.32,8.44,42.82,54.87,145283.0,26400.0,339288.0,-15.65,61.0,X-LC,3.31,2.0,0.18,2.51,17.42,X40,BTT,IT
75,TMPV,600.00,0.40,-17.52,72.39,42.18,165030.0,-48437.0,227973.0,-25.52,31.0,X-LC,6.36,3.0,-0.29,1.69,1.35,XY24,NTT,AUTO
81,VBL,671.64,0.03,-3.31,40.32,35.67,123129.0,-10463.0,305379.0,-14.83,58.0,X-LC,4.63,4.0,-0.08,2.26,10.08,X40N,ATH,FMCG
43,ITC,452.00,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.41,-37.45,112.44,32.89,54304.0,-28910.0,48296.0,-696.43,55.0,L-MC,5.88,259.0,-0.53,0.36,33.00,XR,NTT,BANKS
6,ASIANTILES,137.00,0.80,-15.22,114.30,81.67,90902.0,-14281.0,79529.0,7150.00,49.0,L-SC,13.60,271.0,-0.16,0.59,54.01,XR,NTT,CERAMICS
50,MASFIN,398.61,0.84,-1.21,23.54,22.05,22786.0,-1185.0,96795.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.70,XR,ATH,FINANCE
13,BSOFT,831.70,0.51,-22.02,93.35,50.79,102394.0,-30965.0,109688.0,0.12,67.0,H-SC,5.97,151.0,-0.30,0.81,27.71,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.41,-12.11,13.81,0.02,21186.0,-21146.0,153413.0,133.82,53.0,M-SC,15.85,234.0,-1.00,1.14,33.44,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.51,-22.02,93.35,50.79,102394.0,-30965.0,109688.0,0.12,67.0,H-SC,5.97,151.0,-0.30,0.81,27.71,XR,ATH,IT
1,ABB,7934.0,0.68,-3.16,50.32,45.57,127483.0,-8275.0,253344.0,-37.78,60.0,H-MC,4.78,120.0,-0.06,1.88,7.80,AR,NTT,ELECTRICAL
56,RAJESHEXPO,518.0,-0.05,-62.84,169.31,0.07,86297.0,-86207.0,50970.0,1768.69,58.0,L-SC,2.43,268.0,-1.00,0.38,26.76,OX40N,NTT,JEWELLERY
0,5PAISA,593.0,0.20,-41.34,92.47,12.90,118519.0,-90326.0,128170.0,88.70,43.0,H-SC,11.84,161.0,-0.76,0.95,6.57,OX40N,NTT,FINANCE
84,WIPRO,420.0,0.35,6.71,61.43,72.27,111076.0,11370.0,180818.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.30,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.73
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.34
LC    24.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.33
X40      22.36
X40N     10.87
XR       10.14
XY25      9.40
AR        9.34
OX40N     7.78
SR        1.02
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.63
X-MC    23.55
X-LC    21.07
M-SC    12.11
X-SC     6.89
H-MC     5.03
L-SC     1.43
M-MC     1.40
M-LC     1.34
H-LC     1.16
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.32,-7.72,42.55
IT,12.80,-16.68,79.77
FINANCE,9.62,-19.47,69.95
ELECTRICAL,6.24,-11.61,52.49
MISC,6.07,-58.60,123.57
PAINTS,6.06,-13.05,29.85
INSURANCE,4.61,-1.66,37.48
PHARMA,4.10,-1.01,33.34
AUTO,3.44,-23.97,77.12


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3215638.0,21
AR,1312314.0,10
XR,1294217.0,13
X40,1019869.0,14
X40N,980657.0,9
OX40N,744071.0,10
XY25,373492.0,6
SR,282711.0,2
MH,78466.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3654966.0,25
M-SC,1426804.0,15
X-MC,1303615.0,16
X-SC,989007.0,8
X-LC,921177.0,11
H-MC,400956.0,3
L-SC,268397.0,3
M-LC,111076.0,1
H-LC,77328.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1272791.0      6
           AR         901802.0      5
M-SC       XY24       815170.0      6
H-SC       XR         772983.0      7
X-SC       X40N       507024.0      3
X-MC       X40        451633.0      7
           XY24       398527.0      3
X-LC       X40        382063.0      5
H-SC       OX40N      346213.0      4
M-SC       OX40N      311561.0      5
X-SC       XY24       295810.0      3
H-SC       SR         282711.0      2
X-MC       X40N       272523.0      4
X-LC       XY24       239249.0      2
H-MC       AR         206865.0      2
X-LC       X40N       201110.0      2
H-MC       XY24       194091.0      1
X-SC       X40        186173.0      2
L-SC       XR         182100.0      2
X-MC       XY25       180932.0      2
M-SC       XR         173754.0      2
           AR         126319.0      2
M-LC       XR         111076.0      1
X-LC       XY25        98755.0      2
L-SC       OX40N       86297.0      1
H-SC       MH          78466.0      1
H-LC       AR          77328.0      1
M-MC       XY25        56068.0      1
L-MC       XR          54304.0      1
L-LC       XY25        37737.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
